In [ ]:
!pip install langchain openai unstructured

In [ ]:
import os
import csv
import pandas as pd
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from IPython.display import Markdown, display
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
)

!rm -rf './data'
!rm -f 'result.md'
os.makedirs('data')
os.environ["OPENAI_API_KEY"] = "HERE IS THE API KEY"

human_message_prompt = HumanMessagePromptTemplate(
    prompt=PromptTemplate(
        input_variables=["comments"],
        template="Classify following comments into Technical and Non-technical:\n\n\n{comments}",
    )
)

chat = ChatOpenAI(temperature=0)
chat_prompt_template = ChatPromptTemplate.from_messages([human_message_prompt])

In [ ]:
data = pd.read_csv("test.csv", delimiter=",")

with open('data/file.txt', 'w', encoding='utf-8') as f:
  for row in data['review']:
    f.write(row.split('READ MORE')[0] + '\n')

In [ ]:
# load all documents
loader = DirectoryLoader('data', glob="**/*.txt")
documents = loader.load()

# split the documents into chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 2000,
    chunk_overlap  = 0,
    length_function = len,
)

texts = text_splitter.split_documents(documents)

# initialize the classification model
chain = LLMChain(llm=chat, prompt=chat_prompt_template, verbose=False)

def writeResponse(response):
  with open('result.md', 'a', encoding='utf-8') as f:
    f.write(response)

In [ ]:
for idx, text in enumerate(texts):
  if idx > 1: break

  response = chain.run(text.page_content)
  # writeResponse(response)
  display(Markdown(f"Ans: <b>{response}</b>"))